## Collaborative Filtering

# Objectives
Creating a recommendation system based on collaborative filtering

Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous and can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore recommendation systems based on Collaborative Filtering and implement simple version of one using Python and the Pandas library.

In [1]:
#Importing the libraries
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Acquiring the Data

In [2]:
# reading the downloaded dataset for movies 
movies_df = pd.read_csv("C:/Users/User1/Downloads/ml-25m (2)/ml-25m/movies.csv")
movies_df.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
# reading the downloaded dataset for ratings
ratings_df = pd.read_csv("C:/Users/User1/Downloads/ml-25m (2)/ml-25m/ratings.csv")
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


So each movie has a unique ID, a title with its release year along with it (Which may contain unicode characters) and several different genres in the same field. Let's remove the year from the title column and place it into its own one by using the handy extract function that Pandas has.

Let's remove the year from the title column in movie ID by using pandas' replace function and store it in a new year column.

In [4]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses so only the 4 digits in year is captured
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

C:\Users\User1\AppData\Local\Temp\ipykernel_13968\289546536.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


In [5]:
# Lets look at the result
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [6]:
# lets drop the genres column, since we don't need it  for this recommendation
movies_df = movies_df.drop('genres', axis=1)
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


Lets look at the ratings dataframe

In [7]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. We won't be needing the timestamp column, so let's drop it.

In [8]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', axis=1)
ratings_df.head()

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5


# Collaborative Filtering

Collaborative Filtering
Now it's time to start our work on recommendation systems.

The first technique we're going to take a look at is called Collaborative Filtering, which is also known as User-User Filtering. As hinted by its alternate name, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the Pearson Correlation Function.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/images/User_Item.png" width="800px">


The process for creating a User Based recommendation system is as follows:

Select a user with the movies the user has watched

Based on his rating of the movies, find the top X neighbours

Get the watched movie record of the user for each neighbour

Calculate a similarity score using some formula

Recommend the items with the highest score

Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the userInput. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .




In [9]:
# creating an input user to recommend movies to
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


# Add movieId to input user

With the input complete, let's extract the input movies's ID's from the movies df and add them into it.

We can achieve this by first filtering out the rows that contain the input movies' title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.

In [10]:
#Filtering out the movies by title that can be found in both movies_df and inputMovies,then assigning an inputid variable to
#them
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputId.head()


,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
292,296,Pulp Fiction,1994
1241,1274,Akira,1988
1879,1968,"Breakfast Club, The",1985


In [11]:
#Then merging it to we inputMovies dataframe and assigning a new variable(inputmovies1), so we can get the movieId. 
#It's implicitly merging it by title.
inputMovies1 = pd.merge(inputId, inputMovies)
inputMovies1


,movieId,title,year,rating
0,1,Toy Story,1995,3.5
1,2,Jumanji,1995,2.0
2,296,Pulp Fiction,1994,5.0
3,1274,Akira,1988,4.5
4,164600,Akira,2016,4.5
5,1968,"Breakfast Club, The",1985,5.0


In [12]:
#Dropping information we won't use from the input dataframe and assigning a new variable(inputmovies2)
inputMovies2 = inputMovies1.drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original dataframe or it might spelled differently
# please check capitalisation.
inputMovies2

C:\Users\User1\AppData\Local\Temp\ipykernel_13968\4167752076.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  inputMovies2 = inputMovies1.drop('year', 1)


,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,164600,Akira,4.5
5,1968,"Breakfast Club, The",5.0


The users who has seen the same movies

Now with the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.

In [13]:
#Filtering out users that have watched movies that the input user has watched and storing it in variable userSubset
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies2['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
0,1,296,5.0
70,2,1,3.5
141,2,1968,1.0
254,3,1,4.0
264,3,296,5.0


We now group up the rows by user ID.

In [14]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])
userSubsetGroup.head()

,userId,movieId,rating
0,1,296,5.0
70,2,1,3.5
141,2,1968,1.0
254,3,1,4.0
264,3,296,5.0
...,...,...,...
24999624,162538,1,2.0
24999632,162538,296,2.5
24999661,162538,1968,4.5
24999926,162541,296,3.5


In [15]:
#Let's look at one of the users, e.g. the one in user id=162538
userSubsetGroup.get_group(162538)

,userId,movieId,rating
24999624,162538,1,2.0
24999632,162538,296,2.5
24999661,162538,1968,4.5


Let's also sort these groups so the users that share the most movies in common with the input have higher priority. This provides a richer recommendation since we won't go through every single user.

In [16]:
#Sorting groups so users with movie most in common with the user input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)
#Getting the first 3
userSubsetGroup[0:3]

C:\Users\User1\AppData\Local\Temp\ipykernel_13968\2258376803.py:2: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)


[(43,
        userId  movieId  rating
  5858      43        1     4.0
  5859      43        2     3.5
  5884      43      296     5.0
  5974      43     1274     4.5
  6018      43     1968     4.5),
 (171,
         userId  movieId  rating
  20794     171        1     4.5
  20795     171        2     4.0
  20833     171      296     5.0
  20973     171     1274     4.0
  21047     171     1968     4.0),
 (440,
         userId  movieId  rating
  54640     440        1     3.5
  54641     440        2     2.0
  54669     440      296     5.0
  54756     440     1274     5.0
  54798     440     1968     4.5)]

Similarity of users to input user

Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.
We're going to find out how similar each user is to the input through the Pearson Correlation Coefficient. It is used to measure the strength of a linear association between the two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below.

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y, then, pearson(X, Y) == pearson(X, 2 * Y + 3). This is a pretty important property in recommendation systems because, for example, two users might rate two series of items totally differently in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")


The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation.

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.

We will select a subset of users to iterate through. This limit is imposed because we don't want to waste too much time going through every single user.

userSubsetGroup = userSubsetGroup[0:100]

In [17]:
# Selecting user subset group from 0:100
userSubsetGroup = userSubsetGroup[0:100]




Now, we calculate the Pearson Correlation between input user and subset group, and store it in a dictionary, where the key is the user Id and the value is the coefficient.

In [18]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies3 = inputMovies2.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies3[inputMovies3['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [19]:
pearsonCorrelationDict.items()


dict_items([(43, 0.9460287597114506), (171, 0.3288967572401365), (440, 0.9615384615384616), (597, 0.4688072309384957), (695, 0.8770580193070306), (757, 0.4385290096535153), (772, 0.584705346204686), (860, 0.8594395636904102), (939, 0.716114874039432), (1203, -0.41671450165524815), (1242, 0.711233325153824), (1401, 0.07520710469952328), (1652, 0.537086155529574), (1894, 0.6020183016345586), (1977, -0.323875137815648), (2345, 0.43852900965351493), (2429, 0.4385290096535153), (2469, 0.7161148740394331), (2640, -0.5264497328966636), (2766, -0.21926450482675575), (2982, 0.7307692307692307), (3150, -0.41602514716892186), (3266, 0.657793514480272), (3272, 0.0), (3274, 0.179028718509858), (3624, 0.6698641270570834), (3660, 0.7757911135427185), (3760, 0.5860090386731196), (4006, 0.179028718509858), (4019, 0.39666441401095853), (4060, 0.32328787506992), (4246, 0.89514359254929), (4410, 0.5288858853479448), (4420, 0.657793514480273), (4459, 0.8127665166512523), (4675, 0.537086155529574), (4975, 0

In [20]:
#creating the dataframe with column labels
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.946029,43
1,0.328897,171
2,0.961538,440
3,0.468807,597
4,0.877058,695


In [21]:
# the dataframe above shows the top users with similar interest to the user input

Now let's get the top 50 users that are most similar to the input user

In [22]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
99,0.983092,11495
64,0.970725,7723
55,0.962435,6550
94,0.961678,10960
63,0.961678,7571


Now, let's start recommending movies to the input user.

Rating of selected users to all movies

We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our pearsonDF from the ratings dataframe and then store their correlation in a new column called _similarityIndex". This is achieved below by merging of these two tables.

In [23]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,0.983092,11495,1,3.0
1,0.983092,11495,2,1.0
2,0.983092,11495,6,4.5
3,0.983092,11495,10,4.5
4,0.983092,11495,16,4.5


Now all we need to do is simply multiply the movie rating by its weight (the similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

It shows the idea of all similar users to candidate movies for the input user:

In [24]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.983092,11495,1,3.0,2.949275
1,0.983092,11495,2,1.0,0.983092
2,0.983092,11495,6,4.5,4.423913
3,0.983092,11495,10,4.5,4.423913
4,0.983092,11495,16,4.5,4.423913


In [25]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,39.973168,152.887748
2,39.973168,98.535608
3,12.129644,31.955678
4,2.447945,5.662576
5,11.355650,23.548374


In [29]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average, ie, dividing by the sum of the weight
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.824759,1
2,2.465044,2
3,2.634511,3
4,2.313196,4
5,2.073714,5


Now let's sort it and see the top 10 movies that the algorithm recommended!

In [30]:
recommendation_df1 = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df1.head(10)

,weighted average recommendation score,movieId
movieId,,
70927,5.0,70927
111235,5.0,111235
2931,5.0,2931
6660,5.0,6660
1169,5.0,1169
299,5.0,299
96606,5.0,96606
6583,5.0,6583
320,5.0,320


In [31]:
# Get movie information from movie_df using the movieid column, for those movies id that correspond to the movieid 
# of the recommended first 10 movies(in recommendation_df1)
movies_df.loc[movies_df['movieId'].isin(recommendation_df1.head(10)['movieId'].tolist())]

,movieId,title,year
295,299,Priest,1994
316,320,Suture,1993
1141,1169,American Dream,1990
2839,2931,Time of the Gypsies (Dom za vesanje),1989
6460,6583,"Blood of Heroes, The (Salute of the Jugger, The)",1989
6537,6660,"Red Shoes, The",1948
13693,70927,To Each His Own Cinema (Chacun son cinéma ou C...,2007
16097,84952,Confessions (Kokuhaku),2010
18452,96606,Samsara,2011
21575,111235,Jodorowsky's Dune,2013


In [33]:
# the suggested 10 movies by the algorithm, based on users with similar interest with the user input,is shown in the above
#df